# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-selecionada.csv', sep = '|')
df.head()

,acordao,arquivo,texto,areas
0,297/2016-P,547240.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 010.084/2015-0...,Responsabilidade
1,366/2016-P,549518.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 005.933/2014-5...,Finanças Públicas
2,944/2016-P,554399.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 042.038/2012-0...,Responsabilidade
3,30/2016-P,545010.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 000.742/2014-7...,Direito Processual
4,55/2016-P,544046.txt,;-;;Wania Lucia Pasquarelli do NascimentoTCUWa...,Pessoal


In [2]:
df.shape

(10524, 4)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Competência do TCU,Contrato Administrativo',
       'Competência do TCU,Contrato Administrativo,Responsabilidade',
       'Competência do TCU,Convênio',
       'Competência do TCU,Convênio,Direito Processual,Licitação',
       'Competência do TCU,Convênio,Gestão Administrativa',
       'Competência do TCU,Convênio,Responsabilidade',
       'Competência do TCU,Desestatização',
       'Competência do TCU,Direito Processual',
       'Competência do TCU,Direito Processual,Licitação',
       'Competência do TCU,Direito Processual,Pessoal',
       'Competência do TCU,Direito Processual,Responsabilidade',
       'Competência do TCU,Finanças Públicas',
       'Competência do TCU,Finanças Públicas,Licitação',
       'Competência do TCU,Finanças Públicas,Responsabilidade',
       'Competência do TCU,Gestão Administrativa',
       'Competência do TCU,Licitação',
       'Competência do TCU,Licitação,Pessoal',
       'Competência do TCU,Licitação,Responsabilidade',
   

In [4]:
y = lbArea.transform(df['areas'])
y.shape

(10524, 91)

In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 350000
limite_texto = 40000
dim_vetor = 50

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['texto'])

sequences = tokenizer.texts_to_sequences(df['texto'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 344296 unique tokens.


In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (10524, 40000)


In [7]:
x.shape, y.shape

((10524, 40000), (10524, 91))

In [8]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('../externos/model-50.txt')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 344296
Encontrados no modelo: 115064 = 33.42008039593838


# Treinamento

In [10]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0316 12:50:04.476124 140486501517120 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0316 12:50:04.585225 140486501517120 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0316 12:50:04.587080 140486501517120 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0316 12:50:04.594248 140486501517120 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40000, 50)         17500000  
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               22912     
_________________________________________________________________
dense_1 (Dense)              (None, 91)                11739     
Total params: 17,534,651
Trainable params: 17,534,651
Non-trainable params: 0
_________________________________________________________________
Train on 8419 samples, validate on 2105 samples
Epoch 1/20
 384/8419 [>.............................] - ETA: 2:54:25 - loss: 3.2972 - categorical_accuracy: 0.2240

KeyboardInterrupt: 

In [11]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40000, 50)         17500000  
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               68736     
_________________________________________________________________
dense_2 (Dense)              (None, 91)                11739     
Total params: 17,580,475
Trainable params: 80,475
Non-trainable params: 17,500,000
_________________________________________________________________
Train on 8419 samples, validate on 2105 samples
Epoch 1/20
 928/8419 [==>...........................] - ETA: 2:39:43 - loss: 2.7061 - categorical_accuracy: 0.2769

KeyboardInterrupt: 

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)